In [1]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed, Bidirectional, Conv2D, MaxPooling2D, Flatten, LeakyReLU, AveragePooling2D
from keras.utils import to_categorical
import keras
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import pandas as pd
import librosa
import glob
from sklearn.utils import shuffle
from io import StringIO
import warnings
import gc
import tensorflow as tf
from sklearn.metrics import precision_recall_fscore_support as prfs
from sklearn.metrics import roc_auc_score
import random

ImportError: No module named 'librosa'

In [2]:
from keras import backend as K

def auc(y_true, y_pred):
    auc_arr = tf.metrics.auc(y_true, y_pred)
    auc = (auc_arr[0]+auc_arr[1])/2
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [3]:
warnings.filterwarnings("ignore")

if os.path.exists("../input/birdsong-recognition/test_audio/"):
    n_epochs=200
    fc=None
else:
    n_epochs=50
    fc=100

prob_cutoff = 0.5

prepared_test_str = """row_id,site,audio_id,seconds
bulori/XC128942,bulori/XC128942,bulori/XC128942,5
bulori/XC170988,bulori/XC170988,bulori/XC170988,5
normoc/XC54018,normoc/XC54018,normoc/XC54018,5
normoc/XC62791,normoc/XC62791,normoc/XC62791,5
herthr/XC53784,herthr/XC53784,herthr/XC53784,5
herthr/XC119596,herthr/XC119596,herthr/XC119596,5
brnthr/XC31308,brnthr/XC31308,brnthr/XC31308,5
brnthr/XC53695,brnthr/XC53695,brnthr/XC53695,5
vesspa/XC17095,vesspa/XC17095,vesspa/XC17095,5
vesspa/XC17096,vesspa/XC17096,vesspa/XC17096,5
solsan/XC17025,solsan/XC17025,solsan/XC17025,5
norfli/XC11578,norfli/XC11578,norfli/XC11578,5
lesnig/XC27724,lesnig/XC27724,lesnig/XC27724,5
grycat/XC31058,grycat/XC31058,grycat/XC31058,5
eastow/XC53188,eastow/XC53188,eastow/XC53188,5
aldfly/XC2628,aldfly/XC2628,aldfly/XC2628,5
ameavo/XC99571,ameavo/XC99571,ameavo/XC99571,5
amebit/XC127371,amebit/XC127371,amebit/XC127371,5
amecro/XC51410,amecro/XC51410,amecro/XC51410,5
amegfi/XC17120,amegfi/XC17120,amegfi/XC17120,5
"""

# data parameters
n_birds = 5
mfcc_directory = 'data/mfcc_50/'
train_dir = 'data/train_audio/'
birds = os.listdir(train_dir)
random.shuffle(birds)

birdHalf1 = birds[:int(len(birds)/2)]
birdHalf2 = birds[int(len(birds)/2):]
birds = birds[:n_birds]

trainTable = pd.read_csv("data/train.csv")
trainTable1 = trainTable[trainTable["ebird_code"].isin(birdHalf1)]
# trainTable2 = trainTable[trainTable["ebird_code"].isin(['pinwar','plsvir','prawar','purfin','pygnut','rebmer','rebnut','rebsap','rebwoo'])]
trainTable2 = trainTable[trainTable["ebird_code"].isin(birdHalf2)]

neg2 = list(trainTable1.agg(lambda x: train_dir + x.ebird_code + "/" + x.filename, axis=1).sample(n=100))
neg1 = list(trainTable2.agg(lambda x: train_dir + x.ebird_code + "/" + x.filename, axis=1).sample(n=100))

In [4]:
def load_test_audio(path):
    signal, blah = librosa.load(path, sr=22050)
    if len(signal.shape)==1:
        y = signal
    else:
        y = np.average(signal, axis=1) 
    return signal
    

def getBirdMfcc(bird, n_mfcc=50, filecount=None):
    birddirectory = train_dir + "/" + bird
    birdfiles = [os.path.join(train_dir, bird, birdfile) for birdfile in os.listdir(birddirectory)]
    return getFilesMfcc(birdfiles, n_mfcc=n_mfcc, filecount=filecount)

def getBirdMfccPickle(bird, n_mfcc=50, filecount=None):
    birddirectory = "data/mfcc_50/" + bird + ".pickle"
    mfcc = pickle.load(open(birddirectory,'rb')).T
    return mfcc

def getFilesMfcc(files, n_mfcc=50, filecount=None):
    if filecount!=None:
        files = files[:filecount]
    
    mfcc = []
    for file in files:
        print("reading file: {}".format(file))
        try:
            new_y = load_test_audio(file)
        except:
            continue
        if len(new_y)==0:
            continue
        new_mfcc = librosa.feature.mfcc(n_mfcc=50, y=new_y, sr=22050)
        mfcc.append(new_mfcc)
        del(new_y)
    gc.collect()
    mfcc = np.concatenate(mfcc,axis=1).T
    return mfcc

In [5]:
dataMap = {}
testMap = {}

# birds mfcc calcualtion
for bird in birds:
    bird_mfcc = getBirdMfccPickle(bird, filecount=fc)
    train_samples = int(0.9*bird_mfcc.shape[0])
    test_samples = -1*int(0.1*bird_mfcc.shape[0])
    dataMap[bird] = bird_mfcc[:train_samples,]
    testMap[bird] = bird_mfcc[test_samples:,]

# neg mfcc calculation
bird_mfcc = getFilesMfcc(neg1, filecount=fc)
train_samples = int(0.9*bird_mfcc.shape[0])
test_samples = -1*int(0.1*bird_mfcc.shape[0])
dataMap["neg1"] = bird_mfcc[:train_samples,]
testMap["neg1"] = bird_mfcc[test_samples:,]

reading file: data/train_audio/solsan/XC189628.mp3
reading file: data/train_audio/casfin/XC253943.mp3
reading file: data/train_audio/yetvir/XC179321.mp3
reading file: data/train_audio/amerob/XC189808.mp3
reading file: data/train_audio/yerwar/XC190069.mp3
reading file: data/train_audio/plsvir/XC297500.mp3
reading file: data/train_audio/lewwoo/XC388985.mp3
reading file: data/train_audio/juntit1/XC293943.mp3
reading file: data/train_audio/woothr/XC501239.mp3
reading file: data/train_audio/magwar/XC332613.mp3
reading file: data/train_audio/camwar/XC184370.mp3
reading file: data/train_audio/easblu/XC210869.mp3
reading file: data/train_audio/orcwar/XC388420.mp3
reading file: data/train_audio/snobun/XC397754.mp3
reading file: data/train_audio/bkbmag1/XC195446.mp3
reading file: data/train_audio/gnwtea/XC304648.mp3
reading file: data/train_audio/lotduc/XC208355.mp3
reading file: data/train_audio/belspa2/XC309931.mp3
reading file: data/train_audio/commer/XC440672.mp3
reading file: data/train_aud

In [13]:
def get_model():
    model = Sequential()

    model.add(Conv2D(4, kernel_size=(5, 1), activation='tanh', input_shape=(215, 50, 1)))
    model.add(MaxPooling2D((3, 1), padding='same'))
    model.add(Conv2D(2, kernel_size=(3, 3), activation='tanh'))
    model.add(MaxPooling2D((8,1), strides=(8,1), padding='same'))
#     model.add(AveragePooling2D((9,1), strides=(9,1), padding='same'))
    model.add(Flatten())
    model.add(keras.layers.Dropout(0.9))
    model.add(Dense(1, activation='sigmoid'))

    for layer in model.layers:
        print(layer.output_shape)
        if "conv" in layer.name:
            filters, biases = layer.get_weights()
            print(layer.name, filters.shape)

    adamo=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
    model.compile(loss='binary_crossentropy',
                  optimizer=adamo,
                  metrics=[auc])

    gc.collect()
    return model

In [7]:
def train_generator(bird):
    while True:
        sequence_length = 215
        
        xarr = []
        yarr = []
        
        for k,v in {bird:dataMap[bird], "neg1":dataMap["neg1"]}.items():
            nsamples = v.shape[0]
            size=3
            samples = np.random.randint(0,nsamples-sequence_length+1,size=size)
            if k==bird:
                birdohe = np.array([[1]])
            else:
                birdohe = np.array([[0]])
            for sample in samples:
                xf = v[sample:sample+sequence_length,:]
                lb, ub = xf.min(), xf.max()
                xp, yp = [xf < lb, xf > ub , np.logical_not(np.logical_and(xf < lb , xf > ub))], [0, 1, lambda x: (x-lb)/ub]
                xt = np.piecewise(xf, xp, yp)
                
                xarr.append(xt.reshape(1,sequence_length,50,1))
                yarr.append(birdohe.reshape(1,1))
        x_train = np.concatenate(xarr)
        y_train = np.concatenate(yarr)
        x_train, y_train = shuffle(x_train, y_train)
        yield x_train, y_train

def test_generator(bird, size=1500):
    sequence_length = 215

    xarr = []
    yarr = []

    for k,v in {bird:testMap[bird], "neg1":testMap["neg1"]}.items():
        nsamples = v.shape[0]
        samples = np.random.randint(0,nsamples-sequence_length+1,size=size)
        if k==bird:
            birdohe = np.array([[1]])
        else:
            birdohe = np.array([[0]])
        for sample in samples:
            xf = v[sample:sample+sequence_length,:]
            lb, ub = xf.min(), xf.max()
            xp, yp = [xf < lb, xf > ub , np.logical_not(np.logical_and(xf < lb , xf > ub))], [0, 1, lambda x: (x-lb)/ub]
            xt = np.piecewise(xf, xp, yp)

            xarr.append(xt.reshape(1,sequence_length,50,1))
            yarr.append(birdohe.reshape(1,1))
    x_train = np.concatenate(xarr)
    y_train = np.concatenate(yarr)
    x_train, y_train = shuffle(x_train, y_train)
    return x_train, y_train

x_val,y_val = test_generator(bird)

In [20]:
histories = {}
models = {}

for bird in birds:
    model = get_model()
    my_callbacks = [tf.keras.callbacks.EarlyStopping(patience=25, mode="max", restore_best_weights=True)]
    
    history = model.fit_generator(train_generator(bird), 
                                  steps_per_epoch=150, 
                                  epochs=50, verbose=1, 
                                  validation_data=(x_val, y_val),
                                  callbacks=my_callbacks)
    
    models[bird] = model
    histories[bird] = history
    del(history)
    del(model)

(None, 211, 50, 4)
conv2d_15 (5, 1, 1, 4)
(None, 71, 50, 4)
(None, 69, 48, 2)
conv2d_16 (3, 3, 4, 2)
(None, 9, 48, 2)
(None, 864)
(None, 864)
(None, 1)


ValueError: ('Some keys in session_kwargs are not supported at this time: %s', dict_keys(['callbacks']))

In [19]:
print(keras.__version__)

2.2.4


In [41]:
for bird in birds:
    print(bird)
    
    histroy = histories[bird]
    plt.subplot(2,1,1)
    plt.plot(history.history["loss"])
    plt.plot(history.history["val_loss"])

    plt.subplot(2,1,2)
    plt.plot(history.history["auc"])
    plt.plot(history.history["val_auc"])
    plt.show()
    plt.clf()
    
    model = models[bird]
    x_val,y_val = test_generator(bird,size=5000)
    preds = model.predict(x_val)
    preds[preds > 0.5] = 1
    preds[preds <= 0.5] = 0

    [print(e) for e in prfs(y_val,preds,labels=[1,0])]

norpin
[0.85924191 0.81806184]
[0.807  0.8678]
[0.83230198 0.8421972 ]
[5000 5000]
redhea
[0.88094783 0.91818754]
[0.922  0.8754]
[0.90100655 0.8962834 ]
[5000 5000]
savspa


KeyError: 'savspa'

In [ ]:
bird = birds[0]
model = get_model()
x_val,y_val = test_generator(bird)
history = model.fit_generator(train_generator(bird), steps_per_epoch=150, epochs=100, verbose=1, validation_data=(x_val, y_val))

plt.subplot(2,1,1)
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])

plt.subplot(2,1,2)
plt.plot(history.history["auc"])
plt.plot(history.history["val_auc"])
plt.show()

x_val,y_val = test_generator(bird)
preds = model.predict(x_val)
preds[preds > 0.5] = 1
preds[preds <= 0.5] = 0

prfs(y_val,preds,labels=[1,0])

(None, 211, 50, 4)
conv2d_39 (5, 1, 1, 4)
(None, 71, 50, 4)
(None, 69, 48, 2)
conv2d_40 (3, 3, 4, 2)
(None, 9, 48, 2)
(None, 864)
(None, 864)
(None, 1)
Epoch 1/100
150/150 [==============================] - 13s 88ms/step - loss: 0.6638 - auc: 0.5662 - val_loss: 0.6137 - val_auc: 0.7208
Epoch 2/100
150/150 [==============================] - 10s 66ms/step - loss: 0.6236 - auc: 0.7349 - val_loss: 0.6092 - val_auc: 0.7266
Epoch 3/100
150/150 [==============================] - 10s 64ms/step - loss: 0.5918 - auc: 0.7263 - val_loss: 0.5898 - val_auc: 0.7349
Epoch 4/100
150/150 [==============================] - 11s 72ms/step - loss: 0.6159 - auc: 0.7386 - val_loss: 0.6117 - val_auc: 0.7376
Epoch 5/100
150/150 [==============================] - 11s 75ms/step - loss: 0.5886 - auc: 0.7376 - val_loss: 0.5862 - val_auc: 0.7412
Epoch 6/100
150/150 [==============================] - 11s 74ms/step - loss: 0.5839 - auc: 0.7435 - val_loss: 0.5799 - val_auc: 0.7466
Epoch 7/100
150/150 [=================

(array([0.79154728, 0.75374065]),
 array([0.73666667, 0.806     ]),
 array([0.76312155, 0.77899485]),
 array([1500, 1500], dtype=int64))

In [ ]:
for layer in model.layers:
        print(layer.output_shape)
        if "conv" in layer.name:
            filters, biases = layer.get_weights()
            print(layer.name, filters.shape)
            print(filters)

In [11]:
if os.path.exists("../input/birdsong-recognition/test_audio/"):
    test_dir = "../input/birdsong-recognition/test_audio/"
    testtable = pd.read_csv("../input/birdsong-recognition/test.csv")
else:
    test_dir = "../input/birdsong-recognition/train_audio/"
    testtable = pd.read_csv(StringIO(prepared_test_str))


def load_test_clip(signal, start_time, duration=5):
    sr=22050
    maxl = len(signal)
    
    if duration!=None:
        y = signal[max(0,int(start_time)*22050):min(int(start_time+duration)*22050,maxl)]
    else:
        y = signal[max(0,start_time*22050):]
    
    mfcc_feat = librosa.feature.mfcc(n_mfcc=50, y=y, sr=22050)
    return mfcc_feat.T

def make_prediction(sound_clip):
    predidx = [i for i,each in enumerate(make_probabilities(sound_clip)) if each>prob_cutoff]
    if len(predidx)==0:
        predbirds = "nocall"
    else:
        predbirds = " ".join(list(le.inverse_transform(predidx)))
    
    return predbirds

def make_probabilities(sound_clip):
    mfccdim = sound_clip.shape
    sound_clip = sound_clip.reshape(1,mfccdim[0],mfccdim[1])
    
    pred = model.predict(sound_clip).reshape(mfccdim[0],5)
    
    return np.mean(pred,axis=0)

In [12]:
test_info = testtable.sort_values("audio_id")

preds = []
sound_read = ""
for index, row in test_info.iterrows():
    # Get test row information
    site = row['site']
    try:
        start_time = row['seconds'] - 5
    except:
        start_time = 0
    row_id = row['row_id']
    audio_id = row['audio_id']

    try:
        if sound_read != audio_id:
            sound_audio = load_test_audio(test_dir + audio_id + '.mp3')
            sound_read = audio_id
    except:
        print("exception")
        pred = "nocall"
        preds.append(pred)
        continue
        
    if site == 'site_3':
        sound_clip = load_test_clip(sound_audio, 0, duration=None)
    else:
        sound_clip = load_test_clip(sound_audio, start_time)
            
    
    pred = make_prediction(sound_clip)
    preds.append(pred)
    
testtable = test_info.sort_index()

test_submission = testtable.drop(['site', 'audio_id', 'seconds'], axis = 1) 
test_submission['birds'] = preds

test_submission.to_csv('submission.csv', index = None)

exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception


In [13]:
print(test_submission.head(20))

             row_id   birds
0   bulori/XC128942  nocall
1   bulori/XC170988  nocall
2    normoc/XC54018  nocall
3    normoc/XC62791  nocall
4    herthr/XC53784  nocall
5   herthr/XC119596  nocall
6    brnthr/XC31308  nocall
7    brnthr/XC53695  nocall
8    vesspa/XC17095  nocall
9    vesspa/XC17096  nocall
10   solsan/XC17025  nocall
11   norfli/XC11578  nocall
12   lesnig/XC27724  nocall
13   grycat/XC31058  nocall
14   eastow/XC53188  nocall
15    aldfly/XC2628  nocall
16   ameavo/XC99571  nocall
17  amebit/XC127371  nocall
18   amecro/XC51410  nocall
19   amegfi/XC17120  nocall
